In [1]:
import pypdfium2 as pdfium
import re
import pandas as pd
import pickle
import openai
import time
import os
import numpy as np
from dotenv import load_dotenv
load_dotenv()
openai.api_key = os.getenv('OPENAI_API_KEY')

In [2]:
pdf = pdfium.PdfDocument("../02450_Book.pdf")

text_all = ""
for page in pdf:
    textpage = page.get_textpage()
    text_all += " ".join(textpage.get_text_range().splitlines())

In [9]:
text_sentences = list(re.split('([\.\?\!]) ', text_all))
#text_sentences = [''.join(x) for x in zip(text_sentences[0::2], text_sentences[1::2])]
text_sentences = [s for s in text_sentences if len(s)>10]

In [10]:
text_segments = []
seg = ""
threshold = 1000 #chars
for s in text_sentences:
    # if len(s)>threshold:
    #     print("sentence too long...")
    if len(seg)+len(s)>threshold:
        text_segments.append(seg.strip())
        seg = ""
    else:
        seg+=f' {s}'

In [11]:
text_segments

['Tue Herlau, Mikkel N Schmidt and Morten Mørup Introduction to Machine Learning and Data Mining Lecture notes, Spring 2022, version 1.0 Version for print This document may not be redistributed All rights belongs to the authors and DTU April 5, 2022 Technical University of DenmarkNotation cheat sheet Matlab var Type Size Description X Numeric N × M Data matrix: The rows correspond to N data objects, each of which contains M at\ufffetributes attributeNames Cell array M × 1 Attribute names: Name (string) for each of the M attributes N Numeric Scalar Number of data objects M Numeric Scalar Number of attributes Regression y Numeric N × 1 Dependent variable (output): For each data object, y contains an output value that we wish to predict Classification y Numeric N × 1 Class index: For each data object, y con\ufffetains a class index, yn ∈ {0, 1,  , C − 1}, where C is the total number of classes classNames Cell array C × 1 Class names: Name (string) for each of the C classes',
 'Cross-valid

In [125]:
text_segments_old = text_segments

In [128]:
text_segments = [''.join(x) for x in zip(text_segments_old,text_sentences[1::2])]

In [132]:
text_sentences[1::2][0]

'.'

In [129]:
len(text_segments)

671

In [130]:
max([len(seg) for seg in text_segments])

1002

In [133]:
text_segments[0]

' Tue Herlau, Mikkel N Schmidt and Morten Mørup Introduction to Machine Learning and Data Mining Lecture notes, Spring 2022, version 1.0 Version for print This document may not be redistributed All rights belongs to the authors and DTU April 5, 2022 Technical University of DenmarkNotation cheat sheet Matlab var Type Size Description X Numeric N × M Data matrix: The rows correspond to N data objects, each of which contains M at\ufffetributes attributeNames Cell array M × 1 Attribute names: Name (string) for each of the M attributes N Numeric Scalar Number of data objects M Numeric Scalar Number of attributes Regression y Numeric N × 1 Dependent variable (output): For each data object, y contains an output value that we wish to predict Classification y Numeric N × 1 Class index: For each data object, y con\ufffetains a class index, yn ∈ {0, 1,  , C − 1}, where C is the total number of classes classNames Cell array C × 1 Class names: Name (string) for each of the C classes.'

In [27]:
# Ask ChatGPT to make questions to sections
combinations = []

req_per_min = 0
max_req_per_min = 3
last_time_stamp = time.time()
# For each pair of questions
for i in range(0, len(text_segments)):
    context = text_segments[i]
    
    # Check rate limit
    req_per_min += 1
    while req_per_min>=max_req_per_min:
        time_stamp = time.time()
        if int(time.time()-last_time_stamp) > 60:
            last_time_stamp = time_stamp
            req_per_min = 0
        else:
            time.sleep(10)
        
    # Ask the oracle for label for the context and two questions
    try:
        completion = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",
            temperature=0,
            messages=[
            {"role":"system", "content":"You are a system designed to generate questions that can be answered using ANY part of a provided text, and which are specific to it."},
            {"role": "user", "content": f'Please Generate 5 questions CONTEXT: ```{context}```'}
            ]
        )
    except Exception as e:
        print("OPENAI_ERROR:",str(e))
        continue

    # Parse the response to get labels
    res = completion.choices[0].message.content

    # Split and remove index
    split = res.splitlines()
    questions = [re.sub(r'^\d+\.\s', '', q) for q in split]
    #print(questions)
    
    # If 5 questions are not found, assume something went wrong and skip this iteration
    if len(questions) != 5:
        print(f'[Error] questions not generated for {i}')
        continue
    
    # Save data into a dataframe
    combinations += [{'context': context, 'question': question} for question in questions]
    print(f"Questions from round {i+1} saved!")

Questions from round 1 saved!
Questions from round 2 saved!
Questions from round 3 saved!
Questions from round 4 saved!
Questions from round 5 saved!
Questions from round 6 saved!
Questions from round 7 saved!
Questions from round 8 saved!
Questions from round 9 saved!
Questions from round 10 saved!
Questions from round 11 saved!
Questions from round 12 saved!
Questions from round 13 saved!
Questions from round 14 saved!
Questions from round 15 saved!
Questions from round 16 saved!
Questions from round 17 saved!
Questions from round 18 saved!
Questions from round 19 saved!
Questions from round 20 saved!
Questions from round 21 saved!
Questions from round 22 saved!
Questions from round 23 saved!
Questions from round 24 saved!
Questions from round 25 saved!
Questions from round 26 saved!
OPENAI_ERROR: That model is currently overloaded with other requests. You can retry your request, or contact us through our help center at help.openai.com if the error persists. (Please include the reque

In [74]:
# Ask ChatGPT to make questions to failed sections in 02450
#fails = [26, 56, 136, 153, 163, 199, 205, 320, 337, 358, 395, 424, 432, 463, 568, 599, 626, 650, 669]
#fails = [358, 395]
fails = [358]

req_per_min = 0
max_req_per_min = 2
last_time_stamp = time.time()
# For each pair of questions
for f in fails:
    context = text_segments[f]
    
    # Check rate limit
    req_per_min += 1
    while req_per_min>=max_req_per_min:
        time_stamp = time.time()
        if int(time.time()-last_time_stamp) > 59:
            last_time_stamp = time_stamp
            req_per_min = 0
        else:
            time.sleep(15)
        
    # Ask the oracle for label for the context and two questions
    try:
        completion = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",
            temperature=0,
            messages=[
            {"role":"system", "content":"You are a system designed to generate questions that can be answered using ANY part of a provided text, and which are specific to it."},
            {"role": "user", "content": f'Please Generate 5 questions CONTEXT: ```{context}```'}
            ]
        )
    except Exception as e:
        print("OPENAI_ERROR:",str(e))
        continue

    # Parse the response to get labels
    res = completion.choices[0].message.content

    # Split and remove index
    split = res.splitlines()
    questions = [re.sub(r'^\d+\.\s', '', q) for q in split]
    #print(questions)
    
    # If 5 questions are not found, assume something went wrong and skip this iteration
    if len(questions) != 5:
        print(f'[Error] questions not generated for {f}')
        print(questions)
        continue
    
    # Save data into a dataframe
    fail_combinations += [{'context': context, 'question': question} for question in questions]
    print(f"Questions from round {f} saved!")

Questions from round 358 saved!


In [161]:
#df = pd.DataFrame(columns=['context', 'question'], data = combinations)

with open('./data/final_2450.pkl', 'wb') as fp:
    pickle.dump(df, fp)

In [75]:
df_fail = pd.DataFrame(columns=['context', 'question'], data = fail_combinations)

with open('./data/fails3_2450.pkl', 'wb') as fp:
    pickle.dump(df_fail, fp)

In [4]:
with open('./data/fails3_2450.pkl', 'rb') as f:
    df_fail = pickle.load(f)

In [157]:
# 26, 56, 136, 153, 163, 199, 205, 320, 337, 424, 432, 463, 568, 599, 626, 650, 669, 395, 358
text_segments[359]

' Root mean square error (rmse) for the train\ufffeing and test set when using an artificial neural network with three hidden units to predict the Area of an island based only on the four attributes x1, x2, x5 and x6 using the hold-out method with 40 % of the observations hold-out for testing Attribute description Abbrev x1 Number of plant species Plants x2 Number of endemic plant species E-Plants x3 Area of island (in km2 ) Area x4 Max elevation above sea-level (in m) Elev x5 Distance to nearest island (in km) DistNI x6 Distance to Santa Cruz Island (in km) StCruz x7 Area of adjacent island (in km2 ) AreaNI Table 10.7 The seven attributes of the data on a selection of 29 of the Gal´apagos islands A Neither forward nor backward selection will identify the optimal feature combination for this problem B Forward selection will result in a better model being selected than using backward selection C Backward selection will terminate at the model that includes the features x1, x2, and x6'

In [152]:
df_fail.loc[90,'context']

' D For least squares linear regression the test error will always decrease as we include more attributes in the model E Don’t know Question 4: We would like to fit an artificial neural network (ANN) model to the Galagapos dataset shown in Table 10.7 To reduce the computational cost of fitting the models it was decided to not include Elev, i.e x4, and AreaNI, i.e x7 in the ANN models We there\ufffefore only consider x1, x2, x5 and x6 in order to predict Area, i.e x3 An artificial neural network with three hid\ufffeden units is applied to the data with these four attributes and trained using different combinations of the four at\ufffetributes x1, x2, x5 and x6 Table 10.6 gives the training and test performance in terms of root mean squared error (rmse) of the ANN for different combinations of the con\ufffesidered attributes Which one of the following statements is correct'

In [156]:
df.loc[1745,'context']

' Root mean square error (rmse) for the train\ufffeing and test set when using an artificial neural network with three hidden units to predict the Area of an island based only on the four attributes x1, x2, x5 and x6 using the hold-out method with 40 % of the observations hold-out for testing Attribute description Abbrev x1 Number of plant species Plants x2 Number of endemic plant species E-Plants x3 Area of island (in km2 ) Area x4 Max elevation above sea-level (in m) Elev x5 Distance to nearest island (in km) DistNI x6 Distance to Santa Cruz Island (in km) StCruz x7 Area of adjacent island (in km2 ) AreaNI Table 10.7 The seven attributes of the data on a selection of 29 of the Gal´apagos islands A Neither forward nor backward selection will identify the optimal feature combination for this problem B Forward selection will result in a better model being selected than using backward selection C Backward selection will terminate at the model that includes the features x1, x2, and x6'

In [158]:
for i in range(5):
    n = round(1744.1+i*0.2,1)
    c = i+90
    df.loc[n] = df_fail.loc[c, 'context'], df_fail.loc[c, 'question']
df

,context,question
0.0,"Tue Herlau, Mikkel N Schmidt and Morten Mørup...",What is the purpose of the Introduction to Mac...
1.0,"Tue Herlau, Mikkel N Schmidt and Morten Mørup...",What is the size of the data matrix X?
2.0,"Tue Herlau, Mikkel N Schmidt and Morten Mørup...",What is the difference between the dependent v...
3.0,"Tue Herlau, Mikkel N Schmidt and Morten Mørup...",What is the range of values that the class ind...
4.0,"Tue Herlau, Mikkel N Schmidt and Morten Mørup...",What is the purpose of the attributeNames cell...
...,...,...
1744.1,D For least squares linear regression the tes...,What is the purpose of fitting an artificial n...
1744.3,D For least squares linear regression the tes...,Which attributes were excluded from the ANN mo...
1744.5,D For least squares linear regression the tes...,Which attributes were considered in the ANN mo...
1744.7,D For least squares linear regression the tes...,How many hidden units were used in the ANN model?


In [159]:
df = df.sort_index().reset_index(drop=True)

In [160]:
df

,context,question
0,"Tue Herlau, Mikkel N Schmidt and Morten Mørup...",What is the purpose of the Introduction to Mac...
1,"Tue Herlau, Mikkel N Schmidt and Morten Mørup...",What is the size of the data matrix X?
2,"Tue Herlau, Mikkel N Schmidt and Morten Mørup...",What is the difference between the dependent v...
3,"Tue Herlau, Mikkel N Schmidt and Morten Mørup...",What is the range of values that the class ind...
4,"Tue Herlau, Mikkel N Schmidt and Morten Mørup...",What is the purpose of the attributeNames cell...
...,...,...
3350,"Rex Bryce, James A Hagans, Thomas C Chalmers,...",What is the title of the article published in ...
3351,"Rex Bryce, James A Hagans, Thomas C Chalmers,...","In which year was the article ""On the theory o..."
3352,"Rex Bryce, James A Hagans, Thomas C Chalmers,...","Who are the authors of the article ""Cluster en..."
3353,"Rex Bryce, James A Hagans, Thomas C Chalmers,...",What is the name of the book published by Addi...


## Genarate answers for A/B context length test

In [174]:
import time
import numpy as np
import pickle
import openai
import os
from dotenv import load_dotenv
load_dotenv()
openai.api_key = os.getenv('OPENAI_API_KEY')

In [2]:
with open('../../web/chatta/ab_test/context_length/df_data_original.pkl','rb') as f:
    df_ab = pickle.loads(f.read())

df_ab['answer_short'] = np.nan
df_ab['answer_medium'] = np.nan
df_ab['answer_long'] = np.nan

In [42]:
# Ask ChatGPT to make answers to 3 lengths of contexs

req_per_min = 0
max_req_per_min = 3
last_time_stamp = time.time()
# For each pair of questions
i = 0
for idx,row in df_ab[df_ab['answer_short'].isna()].sample(n=98).iterrows():
    question = row['question']
    context_short = row['short']
    context_medium = row['medium']
    context_long = row['long']
    
    # Check rate limit
    req_per_min += 3
    while req_per_min>=max_req_per_min:
        time_stamp = time.time()
        if int(time.time()-last_time_stamp) > 60:
            last_time_stamp = time_stamp
            req_per_min = 0
        else:
            time.sleep(5)
        
    # Ask the oracle for label for the context and two questions
    try:
        completion = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",
            temperature=0,
            messages=[
                {"role": "user", "content": f'Act as a teacher. A student asks the following question: {question}. Use the following context to answer the question: {context_short}'}
            ]
        )

        answer_short = completion.choices[0].message.content

        df_ab.loc[idx,'answer_short'] = answer_short

        completion = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",
            temperature=0,
            messages=[
                {"role": "user", "content": f'Act as a teacher. A student asks the following question: {question}. Use the following context to answer the question: {context_medium}'}
            ]
        )

        answer_medium = completion.choices[0].message.content

        df_ab.loc[idx,'answer_medium'] = answer_medium

        completion = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",
            temperature=0,
            messages=[
                {"role": "user", "content": f'Act as a teacher. A student asks the following question: {question}. Use the following context to answer the question: {context_long}'}
            ]
        )

        answer_long = completion.choices[0].message.content

        df_ab.loc[idx,'answer_long'] = answer_long

    except Exception as e:
        print("OPENAI_ERROR:",str(e))
        continue
    print(f"Questions from round {i} saved! Index: {idx}")
    i+=1

Questions from round 0 saved! Index: 1663
Questions from round 1 saved! Index: 1744
Questions from round 2 saved! Index: 2174
Questions from round 3 saved! Index: 1101
Questions from round 4 saved! Index: 2126
Questions from round 5 saved! Index: 68
Questions from round 6 saved! Index: 537
Questions from round 7 saved! Index: 3078
Questions from round 8 saved! Index: 1194
Questions from round 9 saved! Index: 2334
Questions from round 10 saved! Index: 1480
Questions from round 11 saved! Index: 2892
Questions from round 12 saved! Index: 2036
Questions from round 13 saved! Index: 2110
Questions from round 14 saved! Index: 2796
Questions from round 15 saved! Index: 3008
Questions from round 16 saved! Index: 2151
Questions from round 17 saved! Index: 2920
Questions from round 18 saved! Index: 1470
Questions from round 19 saved! Index: 1096
Questions from round 20 saved! Index: 2145
Questions from round 21 saved! Index: 553
Questions from round 22 saved! Index: 2923
Questions from round 23 s

In [114]:
q = df_ab[(~df_ab['answer_short'].isna()) & (df_ab['answer_long'].isna())].iloc[0]['question']
ctx = df_ab[(~df_ab['answer_short'].isna()) & (df_ab['answer_long'].isna())].iloc[0]['long']
df_ab[(~df_ab['answer_short'].isna()) & (df_ab['answer_long'].isna())].iloc[0]

IndexError: single positional indexer is out-of-bounds

In [110]:
completion = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",
            temperature=0,
            messages=[
                {"role": "user", "content": f'Act as a teacher. A student asks the following question: {q}. Use the following context to answer the question: {ctx}'}
            ]
        )

ans = completion.choices[0].message.content

In [112]:
df_ab.loc[(~df_ab['answer_short'].isna()) & (df_ab['long'] == ctx), 'answer_long'] = ans

In [128]:
import pickle
with open('../../web/chatta/ab_test/context_length/df_ab_original.pkl','wb') as f:
    f.write(pickle.dumps(df_ab[~df_ab['answer_short'].isna()]))

# Generate new questions for 2450 and ww2 A/B test

In [13]:
import os
import time
import pickle
import numpy as np
import pandas as pd
import openai
from dotenv import load_dotenv
load_dotenv()
openai.api_key = os.getenv('OPENAI_API_KEY')

In [14]:
with open('./data/ww2_long_contexts.pkl','rb') as f:
    contexts = pickle.loads(f.read())

In [57]:
df_history_results_u = df_history_results.loc[df_history_results['question'].drop_duplicates().index]

In [111]:
q = df_2450_results.loc[df_2450_results['question'].duplicated(keep=False),'question'].unique()[4]
df_2450_results.loc[df_2450_results['question']==q,'choice']

41    answer_context
49    answer_context
Name: choice, dtype: object

In [106]:
np.random.randint(0,2)

1

In [58]:
df_history_results_u.shape

(43, 9)

In [109]:
df_2450_results_u.loc[df_2450_results_u['question']==q,'choice'] = 'answer_context'

In [59]:
df_history_results_updated = pd.concat((df_history_results_u,df_history_extra))

In [60]:
df_history_results_updated['question'].unique().shape

(50,)

In [24]:
df_history_results.loc[df_history_results['question'].duplicated(keep=False),:]

,question,correct_context,answer,answer_correct_context,fetched,fetched_time,choice,answer_context,context
11,What is the main focus of the third chapter of...,The third chapter of the pamphlet examines t...,The main focus of the third chapter of the pam...,The main focus of the third chapter of the pam...,True,1686747591,answer,The main focus of the third chapter of the pam...,As we have already seen in the previous sect...
13,What evidence suggests that Hitler was already...,"At the same time, Overy has argued that, by t...",There is no concrete evidence that suggests Hi...,There is evidence to suggest that Hitler was a...,True,1686743885,answer_context,There is no direct evidence in the given conte...,"In the course of 1940, as the German armies ..."
20,What were some of the factors that led to risi...,The more the League of Nations tried to put p...,"Sure, I can help you with that question. The r...",The rising nationalist protests in Japan and t...,True,1686773241,answer_context,The rising nationalist protests in Japan and t...,The more the League of Nations tried to put p...
28,What were the reasons behind the United States...,"Some historians, such as Arnold Offner, in h...",The United States' policy of appeasement towar...,The reasons behind the United States' policy o...,True,1686747523,answer,The United States' policy of appeasement towar...,The onward march of fascism was underlined b...
29,What was Hitler's plan for expanding German te...,A racially strong and expanding German popul...,:\n\nHitler's plan for expanding German territ...,Hitler's plan for expanding German territory w...,True,1686772009,answer,Hitler's plan for expanding German territory w...,The onward march of fascism was underlined b...
30,What was Hitler's ultimate goal in terms of fo...,"However, there still remained some areas of ...",Hitler's ultimate goal in terms of foreign pol...,Hitler's ultimate goal in terms of foreign pol...,True,1686747507,answer,"According to historians, Hitler's ultimate goa...",But it is less clear that a pact with Russia...
33,What was the main argument presented by A.J.P....,Detailed studies began to show that Hitler’s...,A.J.P. Taylor's main argument in his book Orig...,A.J.P. Taylor's main argument in his book Orig...,True,1686745816,answer,I appreciate your question about A.J.P. Taylor...,As we have already seen in the previous sect...
36,What is the main focus of the third chapter of...,The third chapter of the pamphlet examines t...,The main focus of the third chapter of the pam...,NaN,False,0,NaN,The main focus of the third chapter of the pam...,Much of it has focused on the diplomatic even...
37,What was Hitler's plan for expanding German te...,A racially strong and expanding German popul...,:\n\nHitler's plan for expanding German territ...,NaN,False,0,NaN,Hitler's plan for expanding German territory w...,"As we shall see, social tensions, economic w..."
38,What were the reasons behind the United States...,"Some historians, such as Arnold Offner, in h...",The United States' policy of appeasement towar...,NaN,False,0,NaN,The United States' policy of appeasement towar...,The third chapter of the pamphlet examines t...


In [22]:
df_history_results['question'].unique().shape

(43,)

In [38]:
context_sample = [c for c in contexts if c not in df_history_results['context'].unique()]
context_sample = np.random.choice(contexts,4,replace=False)

In [39]:
# Ask ChatGPT to make questions to sections
data = []

req_per_min = 0
max_req_per_min = 3
last_time_stamp = time.time()
# For each pair of questions
for i in range(len(context_sample)):
    context = context_sample[i]
    
    # Check rate limit
    req_per_min += 1
    while req_per_min>=max_req_per_min:
        time_stamp = time.time()
        if int(time.time()-last_time_stamp) > 60:
            last_time_stamp = time_stamp
            req_per_min = 0
        else:
            time.sleep(10)
        
    # Ask the oracle for label for the context and two questions
    try:
        completion = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",
            temperature=0,
            messages=[
            {"role":"system", "content":"You are a system designed to generate a question that can be answered well using a provided context. It should also be possible to answer the question without knowing the context."},
            {"role": "user", "content": f'CONTEXT: ```{context}```'}
            ]
        )
    except Exception as e:
        print("OPENAI_ERROR:",str(e))
        continue

    # Parse the response to get labels
    res = completion.choices[0].message.content
    
    # Save data into a dataframe
    data += [{'correct_context': context, 'question': res}]
    print(f"Questions from round {i} saved!")

Questions from round 0 saved!
Questions from round 1 saved!
Questions from round 2 saved!
Questions from round 3 saved!


In [37]:
df_history_extra

,question,correct_context,answer,answer_correct_context,fetched,fetched_time,choice,answer_context,context
0,What were the key themes in Hitler's thinking ...,(For further discussion of this topic see th...,NaN,NaN,False,0,NaN,NaN,NaN
1,"What is the focus of the book ""The Origins of ...",The Origins of the Second World War 1933–194...,NaN,NaN,False,0,NaN,NaN,NaN
2,What were the fundamental beliefs of the Nazi ...,"Other races, notably eastern ones, could carr...",NaN,NaN,False,0,NaN,NaN,NaN
3,What was the Sudeten German leader's tactic in...,"Schuschnigg took the hint, and on his return...",NaN,NaN,False,0,NaN,NaN,NaN
6,What were the alternative policies towards Ger...,"He further points out that, by early 1939, t...",NaN,NaN,False,0,NaN,NaN,NaN


,question,correct_context,answer,answer_correct_context,fetched,fetched_time,choice,answer_context,context
0,What were the consequences of the Maginot line...,"Further-more, Britain had only skeletal home...",NaN,NaN,False,0,NaN,NaN,NaN
1,What was the anti-comintern pact and how did i...,The onward march of fascism was underlined b...,NaN,NaN,False,0,NaN,NaN,NaN


In [36]:
df_history_extra = df_history_extra[~df_history_extra['question'].isin(df_history_results['question'].values)]

In [43]:
df_history_extra = pd.concat((df_history_extra,df_history_extra_e[~df_history_extra_e['question'].isin(df_history_results['question'].values)].iloc[0:2]))

In [40]:
df_history_extra = pd.DataFrame(columns=df_history_results.columns, data=data)
df_history_extra['fetched'] = False
df_history_extra['fetched_time'] = 0

In [141]:
df = pd.DataFrame(columns=['question','context'],data=data)

In [143]:
with open('./data/history_long_contexts_questions.pkl','wb') as f:
    f.write(pickle.dumps(df))

# Generate answers for A/B test w and w/o context

In [3]:
with open('./data/2450_long_contexts_questions.pkl','rb') as f:
    df_ab = pickle.loads(f.read())

In [166]:
df_ab['answer'] = np.nan
df_ab['answer_context'] = np.nan

In [74]:
# Ask ChatGPT to make answers to 3 lengths of contexs


req_per_min = 0
max_req_per_min = 3
last_time_stamp = time.time()
# For each pair of questions
i = 0
for idx,row in df_ab.iloc[43:].iterrows():
    question = row['question']
    context = row['context']
    
    # Check rate limit
    req_per_min += 1
    while req_per_min>=max_req_per_min:
        time_stamp = time.time()
        if int(time.time()-last_time_stamp) > 60:
            last_time_stamp = time_stamp
            req_per_min = 0
        else:
            time.sleep(5)
        
    # Ask the oracle for label for the context and two questions
    try:
        completion = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",
            temperature=0,
            messages=[
                {"role": "user", "content": f'Act as a teacher. A student asks the following question: ```{question}```. Answer the question with help from the following context: ```{context}```'}
            ]
        )

        answer = completion.choices[0].message.content

        df_ab.loc[idx,'answer_context'] = answer

    except Exception as e:
        print("OPENAI_ERROR:",str(e))
        continue

    # Check rate limit
    req_per_min += 1
    while req_per_min>=max_req_per_min:
        time_stamp = time.time()
        if int(time.time()-last_time_stamp) > 60:
            last_time_stamp = time_stamp
            req_per_min = 0
        else:
            time.sleep(5)
        
    # Ask the oracle for label for the context and two questions
    try:
        completion = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",
            temperature=0,
            messages=[
                {"role": "user", "content": f'Act as a teacher. A student asks the following question: ```{question}```. Answer the question'}
            ]
        )

        answer = completion.choices[0].message.content

        df_ab.loc[idx,'answer'] = answer

    except Exception as e:
        print("OPENAI_ERROR:",str(e))
        continue

    

    print(f"Questions from round {i} saved! Index: {idx}")
    i+=1

Questions from round 0 saved! Index: 43
Questions from round 1 saved! Index: 44
Questions from round 2 saved! Index: 45
Questions from round 3 saved! Index: 46
Questions from round 4 saved! Index: 47
Questions from round 5 saved! Index: 48
Questions from round 6 saved! Index: 49


In [18]:
df_ab['fetched'] = False
df_ab['fetched_time'] = 0
df_ab['choice'] = np.nan
with open('../../web/chatta/ab_test/context_history/saves/df_ab_original.pkl','wb') as f:
    f.write(pickle.dumps(df_ab))

# Find contexts with NN model

In [16]:
import torch
import tensorflow as tf
# Imports
import numpy as np
import pandas as pd
import pickle
import random

# set randomn seed for reproducibility
seed = 23
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
tf.random.set_seed(seed)

from keras.models import Sequential
from keras.layers import Dense
from keras import metrics
from keras.models import load_model

from sentence_transformers import SentenceTransformer
emb_model = SentenceTransformer('sentence-transformers/multi-qa-mpnet-base-dot-v1')

In [17]:
len(contexts[10])

3281

In [22]:
a = emb_model.encode(contexts[10])

In [30]:
b = emb_model.encode(contexts[10][0:-500])
all(a == b)

True

In [29]:
len(contexts[10][0:-500].split(" "))

471

In [28]:
contexts[10][-500:]

'ed his political elevation entirely to the  effects of the depression in Germany, though the evidence does appear to support this  interpretation Clearly the Nazis, under Hitler’s leadership, were sufficiently organized to  have exploited any internal crisis, though whether this would have resulted in such  widespread electoral support, or perhaps have involved a more forceful seizure of power,  we cannot know Would the Weimar system of coalition governments have survived  without the depression'

In [52]:
# Define NN
neural_net = Sequential()
neural_net.add(Dense(768, input_dim=768*2, activation='relu'))
neural_net.add(Dense(384, activation='relu'))
neural_net.add(Dense(1, activation='sigmoid'))

neural_net.compile(loss='binary_crossentropy', optimizer='adam', metrics=[metrics.TruePositives(name='tp'),
                metrics.FalsePositives(name='fp'),
                metrics.TrueNegatives(name='tn'),
                metrics.FalseNegatives(name='fn'),
                metrics.BinaryAccuracy(name='accuracy'),
                metrics.Precision(name='precision'),
                metrics.Recall(name='recall'),
                metrics.AUC(name='auc'),
                metrics.AUC(name='prc', curve='PR')])

In [53]:
neural_net.load_weights('./Results/NN/basic_all_ctx/')

In [54]:
# with open('../../web/chatta/ab_test/context_history/saves/df_ab_original_old.pkl','rb') as f:
#     df_ab = pickle.loads(f.read())
with open('./data/ww2_long_contexts.pkl','rb') as f:
    contexts = pickle.loads(f.read())
with open(f'./Embeddings/ww2_context_embeddings_4000.pkl', 'rb') as f:
    context_embeddings = pickle.loads(f.read())

In [92]:
with open('../../web/chatta/ab_test/context_2450/saves/df_ab_original_old.pkl','rb') as f:
    df_ab = pickle.loads(f.read())
with open('./data/2450_long_contexts.pkl','rb') as f:
    contexts = pickle.loads(f.read())
with open(f'./Embeddings/2450_context_embeddings_4000.pkl', 'rb') as f:
    context_embeddings = pickle.loads(f.read())

In [61]:
df_ab = df_history_results_updated

In [62]:
questions = df_ab['question'].unique()

In [63]:
question_embeddings = {q:emb_model.encode(q) for q in questions}

In [64]:
q_to_found_ctx = {}
for q in questions:
    q_emb = question_embeddings[q]
    inp = np.array([
    np.concatenate((context_embeddings[ctx],q_emb)) for ctx in contexts])
    out = neural_net(inp)
    #out = np.array([[np.dot(context_embeddings[ctx],q_emb)] for ctx in contexts])
    found_ctx = contexts[np.argmax(out)]
    q_to_found_ctx[q] = found_ctx

In [96]:
df_ab.rename(columns={'context':'correct_context'},inplace=True)
df_ab.rename(columns={'answer_context':'answer_correct_context'},inplace=True)
df_ab['answer_context'] = np.nan
df_ab['context'] = np.nan

In [72]:
for idx,row in df_ab.iloc[43:].iterrows():
    df_ab.loc[idx,'context'] = q_to_found_ctx[row['question']]

In [78]:
with open('../../web/chatta/ab_test/context_history/saves/df_ab_fixed.pkl','wb') as f:
    f.write(pickle.dumps(df_ab))

In [2]:
import pickle

In [79]:
with open('../../web/chatta/ab_test/context_history/saves/df_ab_fixed.pkl','rb') as f:
    df_fixed = pickle.loads(f.read())

In [4]:
df_fixed.loc[df_fixed['question']=='What is the main focus of the third chapter of the pamphlet "The Origins of the Second World War 1933-1941"?','correct_context']

11      The third chapter of the pamphlet examines t...
36      The third chapter of the pamphlet examines t...
Name: correct_context, dtype: object

In [12]:
df_fixed['question'].unique().shape

NameError: name 'df_fixed' is not defined

In [4]:
with open('../ab_test_2450_final.pkl','rb') as f:
    df_2450_results = pickle.loads(f.read())
with open('../ab_test_history_final.pkl','rb') as f:
    df_history_results = pickle.loads(f.read())

In [3]:
with open('../16_2450_final.pkl','rb') as f:
    df_2450_results = pickle.loads(f.read())
with open('../16_history_final.pkl','rb') as f:
    df_history_results = pickle.loads(f.read())

In [9]:
df_2450_results['choice'].value_counts()

answer            28
answer_context    22
Name: choice, dtype: int64

In [11]:
df_history_results['choice'].value_counts()

answer            30
answer_context    20
Name: choice, dtype: int64

In [8]:
df_history_results['choice'].value_counts()

answer            33
answer_context    17
Name: choice, dtype: int64

In [11]:
df_2450_results['question'].unique().shape

(44,)

In [14]:
df_2450_results['question'].value_counts()

What is the definition of a symmetric matrix?                                                                                                                      3
What is the impurity measure used in the decision tree in the given context?                                                                                       2
What is the purpose of using a baseline in machine learning?                                                                                                       2
What is the most common choice for the logistic function used to define the density of the Bernoulli distribution?                                                 2
What is the horizon effect and how does pruning attempt to address it?                                                                                             2
What is the main distinction in statistical testing between setup I and setup II?                                                                                  1
What is th

In [15]:
df_history_results['question'].unique().shape

(35,)

In [16]:
df_history_results['question'].value_counts()

What was the significance of the German-Soviet non-aggression pact in the context of the events leading up to World War II?                                                                                3
What were the three developments that split open the united front of Britain, France, and Italy against German rearmament in the mid-1930s?                                                                3
What were the objectives of Japan's expansion in the Pacific region during World War II?                                                                                                                   3
What was Hitler's plan for expanding German territory and how did he justify it?                                                                                                                           2
What are some recommended sources for studying the strategic and economic problems facing Britain in the inter-war period, as well as specific crises that occurred between 1935 and